In [1]:
x = "raw/us_options_opra/day_aggs_v1/2023/01/2023-01-03.csv.gz"
y = "processed/"+'/'.join(x.split('/')[1:]).split('.')[0]+".parquet"
y

'processed/us_options_opra/day_aggs_v1/2023/01/2023-01-03.parquet'

In [8]:
import boto3
import io
import sys
from minio import Minio
from botocore.config import Config
import pandas as pd
import gzip, zlib
from io import BytesIO, StringIO
import asyncio
from concurrent.futures import ThreadPoolExecutor

MINIO_BUCKET = 'polygon'
MINIO_ENDPOINT = 'minio:9000'
MINIO_ACCESS_KEY = 'jared'
MINIO_SECRET_KEY = 'password'
PREFIX = 'us_options_opra/minute_aggs_v1/2024/10'
POLYGON_BUCKET = 'flatfiles'
POLYGON_ACCESS_KEY = '9e922ec1-743d-4916-b10e-ad3d0f3855aa'
POLYGON_SECRET_KEY = 'aFJoRuGWM1sxaaOyaiZBlRPNocTH8uap'
POLYGON_ENDPOINT='https://files.polygon.io'

minio_client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
polygon_session = boto3.Session(
    aws_access_key_id=POLYGON_ACCESS_KEY,
    aws_secret_access_key=POLYGON_SECRET_KEY,
)
polygon_s3_client = polygon_session.client(
    's3',
    endpoint_url=POLYGON_ENDPOINT,
    config=Config(
        signature_version='s3v4',
        max_pool_connections=100
    ),

)
zipped_byte_stream = BytesIO()
parquet_byte_stream = BytesIO()
paginator = polygon_s3_client.get_paginator('list_objects_v2')


for page in paginator.paginate(Bucket=POLYGON_BUCKET, Prefix=PREFIX):
    for obj in page['Contents']:
        print(f"Downloading {obj['Key']}...")
        data = io.BytesIO()
        polygon_s3_client.download_fileobj(
            POLYGON_BUCKET,
            obj['Key'],
            data
        )
        data.seek(0)
        print(f"Uploading {obj['Key']}...")
        minio_client.put_object(
            MINIO_BUCKET,
            "raw/"+obj['Key'],
            data,
            length=-1,
            part_size=1024*1024*5
        ) 
        print(f"{obj['Key']} uploaded successfully")

Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-01.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-01.csv.gz uploaded successfully
Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-02.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-02.csv.gz uploaded successfully
Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-03.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-03.csv.gz uploaded successfully
Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-04.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-04.csv.gz uploaded successfully
Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-07.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-07.csv.gz uploaded successfully
Uploading us_options_opra/minute_aggs_v1/2024/10/2024-10-08.csv.gz...
us_options_opra/minute_aggs_v1/2024/10/2024-10-08.csv.gz uploaded successfully


In [ ]:
# zipped_byte_stream = BytesIO()
# parquet_byte_stream = BytesIO()
# data = minio_client.get_object(
#     bucket_name=MINIO_BUCKET, 
#     object_name="us_options_opra/minute_aggs_v1/2023/01/2023-01-03.csv.gz",
# )
# zipped_byte_stream.write(data.read())
# zipped_byte_stream.seek(0)
# unzipped_file = gzip.decompress(zipped_byte_stream.read())
# df = pd.read_csv(BytesIO(unzipped_file))
# df.to_parquet(parquet_byte_stream, engine="pyarrow")

#print(data.read(128))

for page in paginator.paginate(Bucket=MINIO_BUCKET, Prefix=PREFIX):
   for obj in page['Contents']:
        #print(obj['Key'].split('.')[0]+".parquet")
        print(f"Downloading {obj['Key']}...")
        zipped_byte_stream = io.BytesIO()
        data = minio_client.get_object(
            bucket_name=MINIO_BUCKET, 
            object_name="us_options_opra/minute_aggs_v1/2023/01/2023-01-03.csv.gz",
        )
        zipped_byte_stream.write(data.read())
        zipped_byte_stream.seek(0)
        print(f"Unzipping...")
        unzipped_file = gzip.decompress(zipped_byte_stream.read())
        print(f"Converting csv to parquet...")
        df = pd.read_csv(BytesIO(unzipped_file))
        parquet_byte_stream.write(df.to_parquet(engine="pyarrow"))
        parquet_byte_stream.seek(0)
        print(f"Writing file...")
        minio_client.put_object(
            MINIO_BUCKET,
            "processed/"+obj['Key'].split('.')[0]+".parquet",
            parquet_byte_stream,
            length=-1,
            part_size=1024*1024*5
        )
        print("File uploaded successfully!")

#paginator = polygon_s3_client.get_paginator('list_objects_v2')
#for page in paginator.paginate(Bucket=POLYGON_BUCKET, Prefix=PREFIX):
#    if 'Contents' in page:
#        for obj in page['Contents']:
#            
#            print(f"Processing file: {obj['Key']}")
#            # Download object from Polygon
#            raw_data_file = open("raw_file", 'wb+')
#            polygon_s3_client.download_fileobj(
 #               POLYGON_BUCKET,
 #               obj['Key'],
#                raw_data_file
#            )
#            raw_data_file.close()
#            # decompress
#            unzipped_file = gzip.GzipFile("raw_file", "rb")
#            # convert to parquet
#            df = pd.read_csv(unzipped_file)
#            df.to_parquet(path="parquet_file", engine="pyarrow")

#for page in paginator.paginate(Bucket=POLYGON_BUCKET, Prefix=PREFIX):
#    for obj in page['Contents']:
#        print(obj['Key'])
        #data = io.BytesIO()
        #polygon_s3_client.download_fileobj(
        #    POLYGON_BUCKET,
        #    obj['Key'],
        #    data
        #)
        #data.seek(0)
        #minio_client.put_object(
        #    MINIO_BUCKET,
        #    obj['Key'],
        #    data,
        #    length=-1,
        #    part_size=1024*1024*5
        #)

In [ ]:
# for page in paginator.paginate(Bucket=DATA_BUCKET, Prefix=PREFIX):
#    for obj in page['Contents']:
#         #print(obj['Key'].split('.')[0]+".parquet")
#         print(f"Downloading {obj['Key']}...")
#         zipped_byte_stream = io.BytesIO()
#         polygon_s3_client.download_fileobj(
#            DATA_BUCKET,
#            obj['Key'],
#            zipped_byte_stream
#         )
#         zipped_byte_stream.seek(0)
#         print(f"Unzipping...")
#         unzipped_file = gzip.decompress(zipped_byte_stream.read())
#         print(f"Converting csv to parquet...")
#         df = pd.read_csv(BytesIO(unzipped_file))
#         parquet_byte_stream.write(df.to_parquet(engine="pyarrow"))
#         parquet_byte_stream.seek(0)
#         print(f"Writing file...")
#         minio_client.put_object(
#           MINIO_BUCKET,
#           obj['Key'].split('.')[0]+".parquet",
#           parquet_byte_stream,
#           length=-1,
#           part_size=1024*1024*5
#         )
#         print("File uploaded successfully!")

In [ ]:
# async def transfer_file(obj_key, data_bucket, minio_bucket):
#     data = io.BytesIO()
#     print(f"Downloading {obj_key}...")
#     await asyncio.to_thread(polygon_s3_client.download_fileobj, data_bucket, obj_key, data)
#     data.seek(0)
#     print(f"Uploading {obj_key}...")
#     await asyncio.to_thread(minio_client.put_object, minio_bucket, obj_key, data, length=-1, part_size=1024*1024*5)

# async def process_page(page, data_bucket, minio_bucket):
#     tasks = []
#     for obj in page['Contents']:
#         obj_key = obj['Key']
#         tasks.append(transfer_file(obj_key, data_bucket, minio_bucket))
#     await asyncio.gather(*tasks)

# async def process_all_pages(paginator, data_bucket, minio_bucket):
#     for page in paginator.paginate(Bucket=data_bucket, Prefix=PREFIX):
#         await process_page(page, data_bucket, minio_bucket)

# def main():
#     # Use your paginator here
#     process_all_pages(paginator, DATA_BUCKET, MINIO_BUCKET)

# main()

In [ ]:
objects = minio_client.list_objects(MINIO_BUCKET, recursive=True, prefix="raw/us_options_opra/day_aggs_v1/2023/01")
for obj in objects:
    print(f"Downloading {obj.object_name}...")
    zipped_byte_stream = io.BytesIO()
    data = minio_client.get_object(
        bucket_name=MINIO_BUCKET, 
        object_name=obj.object_name,
    )
    zipped_byte_stream.write(data.read())
    zipped_byte_stream.seek(0)
    print(f"Unzipping...")
    unzipped_file = gzip.decompress(zipped_byte_stream.read())
    print(f"Converting csv to parquet...")
    df = pd.read_csv(BytesIO(unzipped_file))
    parquet_byte_stream.write(df.to_parquet(engine="pyarrow"))
    parquet_byte_stream.seek(0)
    print(f"Destination file: {'processed/'+obj.object_name.split('.')[0]+'.parquet'}")
    #print(f"Writing file...")
    #minio_client.put_object(
    #    MINIO_BUCKET,
    #    "processed/"+obj.object_name.split('.')[0]+".parquet",
    #    parquet_byte_stream,
    #    length=-1,
    #    part_size=1024*1024*5
    #)
    #print("File uploaded successfully!")